In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import load_img  # Correct import for Keras image loading


In [2]:
pwd

'C:\\Users\\User\\minor_project'

In [3]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    image = cv2.resize(image, (128, 128))  # Resize to match input shape
    image = image / 255.0  # Normalize pixel values (0-1)
    image = np.expand_dims(image, axis=-1)  # Add channel dimension for CNN
    return image

In [4]:
IMG_SIZE = 128
BATCH_SIZE = 32

In [5]:
DATASET_PATH = r"C:\Users\User\minor_project\dataset"

In [6]:
labels_dict = {}
for country in os.listdir(DATASET_PATH):
    country_path = os.path.join(DATASET_PATH, country)
    if os.path.isdir(country_path):  # Check if it's a folder
        for letter in os.listdir(country_path):
            label_key = f"{country}/{letter}"
            label_value = f"{country}_{letter}"
            labels_dict[label_key] = label_value


In [7]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 47912 images belonging to 5 classes.
Found 11977 images belonging to 5 classes.


In [8]:
print(train_data.class_indices.keys())

dict_keys(['America', 'Filipino', 'India', 'Indonesia', 'Malaysia'])


In [9]:
print(labels_dict.keys())

dict_keys(['America/a', 'America/b', 'America/c', 'America/d', 'America/e', 'America/f', 'America/g', 'America/h', 'America/i', 'America/j', 'America/k', 'America/l', 'America/m', 'America/n', 'America/o', 'America/p', 'America/q', 'America/r', 'America/s', 'America/t', 'America/u', 'America/v', 'America/w', 'America/x', 'America/y', 'America/z', 'Filipino/A', 'Filipino/B', 'Filipino/C', 'Filipino/D', 'Filipino/E', 'Filipino/F', 'Filipino/G', 'Filipino/H', 'Filipino/I', 'Filipino/J', 'Filipino/K', 'Filipino/L', 'Filipino/M', 'Filipino/N', 'Filipino/O', 'Filipino/P', 'Filipino/Q', 'Filipino/R', 'Filipino/S', 'Filipino/T', 'Filipino/U', 'Filipino/V', 'Filipino/W', 'Filipino/X', 'Filipino/Y', 'Filipino/Z', 'India/A', 'India/B', 'India/C', 'India/D', 'India/E', 'India/F', 'India/G', 'India/H', 'India/I', 'India/J', 'India/K', 'India/L', 'India/M', 'India/N', 'India/O', 'India/P', 'India/Q', 'India/R', 'India/S', 'India/T', 'India/U', 'India/V', 'India/W', 'India/X', 'India/Y', 'India/Z', '

In [10]:
# # Update the class indices to match 'America_A', 'America_B', etc.
# train_data.class_indices = {labels_dict[key]: value for key, value in train_data.class_indices.items()}
# train_data.class_indices = {labels_dict.get(key, key): value for key, value in train_data.class_indices.items()}
train_data.class_indices= list(labels_dict.keys())
val_data.class_indices = list(labels_dict.keys())

In [11]:
print(train_data.class_indices)

['America/a', 'America/b', 'America/c', 'America/d', 'America/e', 'America/f', 'America/g', 'America/h', 'America/i', 'America/j', 'America/k', 'America/l', 'America/m', 'America/n', 'America/o', 'America/p', 'America/q', 'America/r', 'America/s', 'America/t', 'America/u', 'America/v', 'America/w', 'America/x', 'America/y', 'America/z', 'Filipino/A', 'Filipino/B', 'Filipino/C', 'Filipino/D', 'Filipino/E', 'Filipino/F', 'Filipino/G', 'Filipino/H', 'Filipino/I', 'Filipino/J', 'Filipino/K', 'Filipino/L', 'Filipino/M', 'Filipino/N', 'Filipino/O', 'Filipino/P', 'Filipino/Q', 'Filipino/R', 'Filipino/S', 'Filipino/T', 'Filipino/U', 'Filipino/V', 'Filipino/W', 'Filipino/X', 'Filipino/Y', 'Filipino/Z', 'India/A', 'India/B', 'India/C', 'India/D', 'India/E', 'India/F', 'India/G', 'India/H', 'India/I', 'India/J', 'India/K', 'India/L', 'India/M', 'India/N', 'India/O', 'India/P', 'India/Q', 'India/R', 'India/S', 'India/T', 'India/U', 'India/V', 'India/W', 'India/X', 'India/Y', 'India/Z', 'Indonesia/

In [12]:
import json

# Save the class labels after training
with open("class_indices.json", "w") as f:
    json.dump(train_data.class_indices, f)


In [13]:
num_classes = len(train_data.class_indices)

In [26]:
# input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# x = Conv2D(32, (3,3), activation='relu', padding='same')(input_layer)
# x = MaxPooling2D(pool_size=(2,2))(x)

# x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)

# x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)


In [28]:
output_layer = Dense(num_classes, activation='softmax', name="classification_output")(x)

In [32]:
model = Model(inputs=base_model.input, outputs=output_layer)

In [34]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [36]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


In [41]:
model.fit(train_data, epochs=10, steps_per_epoch=len(train_data), validation_data=val_data, validation_steps=len(val_data),callbacks=[early_stopping, reduce_lr])


Epoch 1/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 2383s 2s/step - accuracy: 0.9803 - loss: 0.0827 - val_accuracy: 0.7524 - val_loss: 1.6611 - learning_rate: 0.0010
Epoch 2/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 2657s 2s/step - accuracy: 0.9950 - loss: 0.0194 - val_accuracy: 0.9179 - val_loss: 0.5419 - learning_rate: 0.0010
Epoch 3/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 2403s 2s/step - accuracy: 0.9943 - loss: 0.0230 - val_accuracy: 0.6415 - val_loss: 57.9779 - learning_rate: 0.0010
Epoch 4/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 2252s 2s/step - accuracy: 0.9949 - loss: 0.0238 - val_accuracy: 0.9588 - val_loss: 0.7109 - learning_rate: 0.0010
Epoch 5/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 2252s 2s/step - accuracy: 0.9878 - loss: 0.0562 - val_accuracy: 0.9501 - val_loss: 0.4735 - learning_rate: 0.0010
Epoch 6/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 2306s 2s/step - accuracy: 0.9960 - loss: 0.0144 - val_accuracy: 0.9911 - val_loss: 0.0517 - learning_rate: 0.0010
Epoch 7/10
1498/1498 ━━━━━━━━━━━━━━━━━━━━ 2448s 2s/step - accur

In [50]:
model.save("sign_language_model.h5")